In [12]:
import sys
sys.path.append('../src')
from stocks import stocks
import pandas as pd
import yfinance as yf
import time
from curl_cffi import requests


In [13]:
session = requests.Session(impersonate="chrome")

In [ ]:
# Daily Closing Price 
data = pd.DataFrame()
for ticker in stocks:
    #print(f"Downloading {ticker}...")
    temp = yf.download(ticker, session=session, period='10y')['Close']
    data[ticker] = temp
    data.to_csv('../data/close.csv')
    time.sleep(2)  # Wait 2 seconds between requests

In [ ]:
ticker_obj = yf.Ticker('AAPL')
data = ticker_obj.info['marketCap']
data